# Matching records extracted from text to BLL01 Incunabula

Using VM's ss where she matched our extracted records with BLL01

In [ ]:
import sys
if "../" not in sys.path:
    sys.path.append("../")
import glob
import pandas as pd

from src.data.reimport_utils import reconstruct_xmls

In [ ]:
entry_paths = sorted(glob.glob("../data/processed/*/catalogue_entries_v1.4.csv"), key=lambda x: int(x.split("BMC_")[1].split("\\")[0]))

In [ ]:
entry_dfs = [pd.read_csv(x, converters={"xmls": reconstruct_xmls}, encoding="utf8") for x in entry_paths]

In [ ]:
ocr_records_df = pd.concat(entry_dfs)[["xmls", "vol_entry_num", "shelfmark", "entry_text"]].reset_index(drop=True)
ocr_records_df["entry_text_spaces"] = ocr_records_df["entry_text"].str.replace("\n", " ")
# ocr_records_df = pd.read_csv("..\\data\\processed\\all_volume_catalogue_entries_v1.0.csv", encoding='latin-1', index_col="Column1")
bll01_index_df = pd.read_csv("..\\data\\processed\\bll01_index.csv", encoding='utf-8')
bll01_index_df.rename(columns={'British Library shelfmark (852 $j)': "bll01_shelfmark", 'Record IDs (001)': "record_id"}, inplace=True)

In [ ]:
ocr_records_df

In [ ]:
# matched_records_df = pd.read_csv("..\\data\\processed\\bll_matching_entries.csv", encoding='latin-1', index_col="index to column1").sort_index().iloc[:-1]

In [ ]:
# matched_records_df = matched_records_df.set_index(matched_records_df.index.astype(int)).rename(columns={"shelfmark": "raw_shelfmark", "normalized shelfmark": "norm_shelfmark"})
# matched_records_df = matched_records_df.join(ocr_records_df.loc[:,["vol_entry_num", "xmls"]], how="left", rsuffix="_ocr")
# matched_records_df.insert(loc=3, column="vol_entry_num", value=matched_records_df["vol_entry_num"] + 1)
# matched_records_df.drop(columns="volume_entry_num", inplace=True)

In [ ]:
ocr_records_df.shape, bll01_index_df.shape, # matched_records_df.shape

In [ ]:
ocr_records_df.head()

Going to stop using `norm_shelfmark` from VM's ss. This is `normalized shelfmark` in `bll_matching_entries.csv`. Going to match instead on `shelfmark` from the combined `catalogue_entries_v1.4.csv`, which is the version produced using the latest entry extraction algorithm.

In [ ]:
ocr_shelfmarks = ocr_records_df["shelfmark"].str.strip()
# ocr_shelfmarks = matched_records_df["norm_shelfmark"].str.strip()

In [ ]:
bll01_index_df["bll01_shelfmark"] = bll01_index_df["bll01_shelfmark"].str.strip()

In [ ]:
bll_ocr_df = bll01_index_df.merge(ocr_shelfmarks.drop_duplicates(), left_on="bll01_shelfmark", right_on="norm_shelfmark", how="left").rename(columns={"normalized_shelfmark": "normalized_ocr_shelfmark"})
# bll_ocr_df = bll01_index_df.merge(ocr_shelfmarks, left_on="bll01_shelfmark", right_on="normalized_shelfmark", how="right").rename(columns={"normalized_shelfmark": "normalized_ocr_shelfmark"})

In [ ]:
bll01_index_df

In [ ]:
ocr_shelfmarks

In [ ]:
bll_ocr_df

In [ ]:
bll_ocr_df.shape

In [ ]:
# ocr_bll_df.to_csv("..\\data\\processed\\ocr_bll.csv", index=False)
# bll_ocr_df.to_csv("..\\data\\processed\\bll_ocr.csv", index=False)

### Creating Tkb page col lookup

Adding a column that converts the xml name to a page number in the Tkb collection

In [ ]:
# all the xml ids present on tkb - if the code didn't find a shelfmark in an xml it won't be represented in matched_records
# use the complete list to convert our incomplete list into page numbers on tkb
with open("..\\data\\interim\\all_xml_ids.txt", "r") as f:
    xml_ids = [x.strip("\n") for x in f.readlines()]

In [ ]:
xml_ids[:5]

In [ ]:
# check all our xmls are present in the complete list
set(ocr_records_df["xmls"].sum()) <= set(xml_ids)
# matched_records_df["xml"].apply(lambda x: x in xml_ids).all()

In [ ]:
all_xml_df = pd.DataFrame({"xml":xml_ids})

all_xml_df["vol"] = all_xml_df["xml"].str.split("_").apply(lambda x: int(x[4]))
all_xml_df["raw_tkb_page"] = all_xml_df["xml"].str.split("_").apply(lambda x: int(x[5]))
all_xml_df["num_cols"] = all_xml_df["xml"].str.split("_").apply(lambda x: int(x[6]))
all_xml_df.sort_values(by=["vol", "num_cols", "raw_tkb_page"], inplace=True)
all_xml_df["tkb_page"] = all_xml_df.groupby(by=["vol", "num_cols"])["raw_tkb_page"].transform(lambda x: (x.diff() > 0).cumsum() + 1)
all_xml_df = all_xml_df.set_index("xml")

In [ ]:
all_xml_df.query("vol == 9 & num_cols == 4").head()

In [ ]:
# all_xml_df.to_csv("..\\data\\processed\\xml_tkb_conversion_table.csv")

### Using Tkb page col lookup

In [ ]:
all_xml_df = pd.read_csv("..\\data\\processed\\xml_tkb_conversion_table.csv", index_col=0)

In [ ]:
vol = pd.merge(ocr_records_df["xmls"].apply(lambda x: x[0]), all_xml_df.drop(columns="raw_tkb_page"), how="left", left_on="xmls", right_on="xml")["vol"]
xmls_long = pd.merge(ocr_records_df["xmls"].explode().reset_index(), all_xml_df.drop(columns="raw_tkb_page"), how="left", left_on="xmls", right_on="xml")
xmls_long["col_pages"] = xmls_long["num_cols"].astype(str) + "-" + xmls_long["tkb_page"].astype(str) + ", "
col_pages = xmls_long.groupby(by="index")["col_pages"].sum().str.strip(" ")

In [ ]:
ocr_records_df["vol"] = vol
ocr_records_df["col_pages"] = col_pages

In [ ]:
matched_records_merged_df = ocr_records_df.merge(right=bll01_index_df, how="left", left_on="shelfmark", right_on="bll01_shelfmark")
matched_records_merged_df = matched_records_merged_df[[
    'xmls', 'vol', 'col_pages', 'vol_entry_num', 'shelfmark', 'bll01_shelfmark', 'record_id', 'uncaptured_sm', 'entry_text', "entry_text_spaces"
]]

In [ ]:
matched_records_merged_df

### Export

In [ ]:
# matched_records_merged_df.to_csv("..\\data\\processed\\incu_ocr_bll_matching_v1.4.csv", encoding="utf-8-sig")

Then add these sheets manually
- BLL01 Index (the complete index of Incunabula in BLL01)
- Missed BLL01 Shelfmarks (Recording for missing shelfmarks and the reason for them being missing)
- Workflow (documentation of the workflow)
- Data Dictionary (explanation of columns)

### Label matched extracted shelfmarks

Helpful in the matching spreadsheet to have a column against the complete list of BLL01 Incunabula shelfmarks that marks which ones have been succesfully extracted by the detection algorithm.

In [ ]:
found_by = pd.Series(1, index=matched_records_merged_df["bll01_shelfmark"].dropna()).rename("found_by_algorithm")

In [ ]:
found_by.shape

In [ ]:
bll01_index = pd.read_csv("../data/processed/bll01_index.csv", encoding="utf8", index_col=0)

In [ ]:
# this must be true otherwise we've lost shelfmarks somewhere
assert found_by.shape == bll01_index.join(found_by)["found_by_algorithm"].dropna().shape

In [ ]:
bll01_index = bll01_index.join(found_by)
bll01_index["found_by_algorithm"] = bll01_index["found_by_algorithm"].fillna(0).astype(int)

In [ ]:
bll01_index.to_csv("../data/processed/bll01_index.csv", encoding="utf8")

### Preserving VMs shelfmark extraction in compressed form

In [ ]:
compressed_shelfmark_rec_ids = matched_records_df.loc[:, "Shelfmark 1":].agg(
    lambda x: [[y, z] for y, z in zip(x[::2], x[1::2]) if type(y) == str], axis=1
).rename("shelfmarks_in_entry_text")

In [ ]:
compressed_shelfmark_rec_ids
# matched_records_tkb_page_df["vm_shelfmark_rec_id_matching"] = compressed_shelfmark_rec_ids

In [ ]:
# check we've as many shelfmarks in the compressed as non-compressed
counts = matched_records_df.loc[:, "Shelfmark 1":].agg(lambda x: x.dropna().shape[0])
counts.iloc[::2].sum(), len(compressed_shelfmark_rec_ids.sum())

In [ ]:
# check we've as many record ids in the compressed as the non-compressed
# not quite correct but then not really using this data anyway
counts.iloc[1::2].sum(), len([x[1] for x in compressed_shelfmark_rec_ids.sum() if type(x[1]) == str])